## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-09-22-50-14 +0000,nypost,"Naked madman breaks Vietnam veterans’ legs, bu...",https://nypost.com/2025/11/09/us-news/naked-ma...
1,2025-11-09-22-49-09 +0000,nyt,Sean Duffy Juggles Shutdown Roles on Flight Sa...,https://www.nytimes.com/2025/11/09/us/politics...
2,2025-11-09-22-43-18 +0000,nypost,Top NYC rabbi issues dire warning on Mamdani: ...,https://nypost.com/2025/11/09/us-news/nycs-chi...
3,2025-11-09-22-42-04 +0000,nyt,"Federal Judge, Warning of ‘Existential Threat’...",https://www.nytimes.com/2025/11/09/us/politics...
4,2025-11-09-22-29-43 +0000,nyt,Trump Officials Push States to ‘Undo’ Work to ...,https://www.nytimes.com/2025/11/09/business/tr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2356/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,31
14,shutdown,13
196,new,11
130,bbc,9
35,states,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
67,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...,77
78,2025-11-09-18-12-48 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...,69
52,2025-11-09-19-59-01 +0000,startribune,Trump administration demands states 'undo' ful...,https://www.startribune.com/trump-administrati...,68
177,2025-11-09-02-00-00 +0000,wsj,Big companies run into a roadblock to getting ...,https://www.wsj.com/politics/policy/trump-bide...,68
58,2025-11-09-19-36-13 +0000,cbc,Trump administration wants U.S. states to 'und...,https://www.cbc.ca/news/world/trump-administra...,65


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
67,77,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...
177,37,2025-11-09-02-00-00 +0000,wsj,Big companies run into a roadblock to getting ...,https://www.wsj.com/politics/policy/trump-bide...
91,32,2025-11-09-15-33-10 +0000,nypost,Lefty Gen-Z NYC Councilman Chi Osse gears up f...,https://nypost.com/2025/11/09/us-news/nyc-coun...
73,32,2025-11-09-18-33-00 +0000,nypost,BBC head Tim Davie resigns over doctored foota...,https://nypost.com/2025/11/09/world-news/bbc-h...
134,32,2025-11-09-10-26-24 +0000,cbc,Super Typhoon Fung-wong hits Philippines after...,https://www.cbc.ca/news/world/super-typhoon-th...
7,28,2025-11-09-22-16-34 +0000,nyt,"Hamas Hands Over Body of Hadar Goldin, Israeli...",https://www.nytimes.com/2025/11/09/world/middl...
149,27,2025-11-09-09-00-00 +0000,cbc,NDP MP ‘very confident’ no New Democrats will ...,https://www.cbc.ca/news/politics/ndp-rejects-f...
79,25,2025-11-09-18-03-36 +0000,nypost,"Crew member dead, 2 critically injured in medi...",https://nypost.com/2025/11/09/us-news/crew-mem...
83,25,2025-11-09-17-30-00 +0000,wsj,President Trump suggests tariff revenue could ...,https://www.wsj.com/politics/policy/trump-sugg...
90,25,2025-11-09-15-56-32 +0000,nypost,Transportation secretary gives dire warning on...,https://nypost.com/2025/11/09/us-news/transpor...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
